In [1]:
# import required packages
import pandas as pd
import numpy as np

import seaborn as sns

from sklearn.metrics import precision_recall_fscore_support, classification_report, confusion_matrix

import warnings
import requests
from requests import get
import random

from mycbr_py_api import MyCBRRestApi as mycbr

In [2]:
x_cols = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'color']

label = 'quality'

In [3]:
# API connection

server = 'localhost'
#server = 'user@hv-6066.idi.ntnu.no'

port = '8080'
base_url = 'http://' + server + ':' + port + '/'

headers = {'Content-type':'application/json'}

obj = mycbr(base_url)


concept = 'wine'

In [4]:
# Deletes all cases from specified concept 
def delete_instances_from_concept(c):
    response = requests.delete('http://localhost:8080/concepts/'+c+'/cases')
    print("cases deleted from " + c + ": " + str(response.ok))
    
# Adds a new casebase 
def put_new_cb(newCbName):
    requests.put(base_url + 'casebases/' + newCbName)
    
# Returns casebase list
def get_casebases():
    raw = pd.DataFrame(requests.get(base_url + 'casebases/').json())
    casebases = pd.DataFrame.from_records(raw).values.tolist()
    return casebases

# add cases to casebase
def add_rows_as_cases(x, c, cb):
    case_id = 'case_' + str(x['index'])
    #x = x.drop(['Target', 'index'])
    x = x.drop(['index'])
    requests.post(base_url + 'concepts/' + c + '/casebases/' + cb +'/cases/' + case_id, data = str(x.to_json()), headers=headers)

# add cases to casebase from dataframe
def add_cases_from_df(df, c, cb):  
    tmp = df.copy(deep=True)
    tmp.reset_index(inplace=True)
    tmp.apply(add_rows_as_cases, args=(c, cb), axis=1)
    
# Add new amalgamation function
def newAmalgamationFunc(c, amalFuncID, amalFuncType, json):
    return requests.put(base_url + 'concepts/' + c + '/amalgamationFunctions?amalgamationFunctionID=' + amalFuncID + '&amalgamationFunctionType=' + amalFuncType + '&attributeWeightsJSON=' + json)


In [5]:
print('Casebase names: ', get_casebases())

Casebase names:  [['cb_class1'], ['cb_class0'], ['cb_class2']]


In [6]:
delete_instances_from_concept(concept)

cases deleted from wine: True


In [7]:
train_class0 = pd.read_csv("./data/train_c0.csv").astype('float64')
train_class1 = pd.read_csv("./data/train_c1.csv").astype('float64')
train_class2 = pd.read_csv("./data/train_c2.csv").astype('float64')


# Adding cases to respective casebases
add_cases_from_df(train_class0, concept, 'cb_class0')    
add_cases_from_df(train_class1, concept, 'cb_class1')    
add_cases_from_df(train_class2, concept, 'cb_class2')    

In [8]:
# Viewing the cases in a given casebase
obj.getAllCasesFromCaseBase(concept, 'cb_class0').head()
# obj.getAllCasesFromCaseBase(concept, 'cb_class1').head()
# bj.getAllCasesFromCaseBase(concept, 'cb_class2').head()

,similarity,caseID,alcohol,chlorides,citric_acid,color,density,fixed_acidity,free_sulfur_dioxide,pH,residual_sugar,sulphates,total_sulfur_dioxide,volatile_acidity
0,1.0,case_170.0,9.2,0.067,0.26,1.0,0.99838,7.5,47.0,3.04,14.95,0.49,178.0,0.29
1,1.0,case_82.0,9.8,0.054,0.37,1.0,0.99795,8.0,23.0,3.32,9.6,0.47,159.0,0.23
2,1.0,case_86.0,9.2,0.06,0.14,1.0,0.99344,6.1,27.0,3.07,3.9,0.34,113.0,0.38
3,1.0,case_63.0,12.5,0.04,0.46,1.0,0.9906,7.2,3.0,3.04,5.0,0.53,29.0,0.31
4,1.0,case_40.0,12.4,0.033,0.42,1.0,0.9911,8.3,18.0,3.2,1.15,0.32,96.0,0.33


In [9]:
# Read and cast shap values to string for creating json objects (for newAmalgamationFunc)
s0 = pd.read_csv("./shap_val/shap0.csv", index_col='col_name').iloc[:-1]
str_map0 = str(s0.to_dict()['shap']).replace("'",'"')

s1 = pd.read_csv("./shap_val/shap1.csv", index_col='col_name').iloc[:-1]
str_map1 = str(s1.to_dict()['shap']).replace("'",'"')

s2 = pd.read_csv("./shap_val/shap2.csv", index_col='col_name').iloc[:-1]
str_map2 = str(s2.to_dict()['shap']).replace("'",'"')

In [10]:
# Set amalgamation functions for each casebase
newAmalgamationFunc(concept,'amal_func_class0', 'WEIGHTED_SUM', str_map0)
newAmalgamationFunc(concept,'amal_func_class1', 'WEIGHTED_SUM', str_map1)
newAmalgamationFunc(concept,'amal_func_class2', 'WEIGHTED_SUM', str_map2)

<Response [200]>